In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session




--------------------------------------------------
## Data Set infomation
Đây là bộ data của 500 khách hàng của 1 sàn thương mại điện tử cung cấp cho chúng ta một số thông tin, bao gồm:

- Avg. Session Length: Độ dài trung bình của một phiên khách hàng truy cập app/website (phút)
- Time on App: Thời gian trung bình của khách hàng truy cập trên App (phút)
- Time on Website: Thời gian trung bình của khách hàng truy cập trên website (phút)
- Length of Membership: Số năm khách hàng sử dụng thẻ thành viên khi mua hàng
- Yearly amount Spent: Tổng số tiền khách hàng mua hàng trong một năm

In [ ]:
data=pd.read_csv('../input/ecommerce-customers/Ecommerce Customers.csv')
data.info()

-------------------------------------------------------------
## TODO 1: EDA

### 1. Univariate Analysis:
#### a. Trình bày các quan sát về phân phối của các biến


In [ ]:
# Histograms for Avg. Session Length
char1=sns.displot(data['Avg. Session Length'])
char2=sns.displot(data['Time on App'])
char3=sns.displot(data['Time on Website'])
char4=sns.displot(data['Length of Membership'])
char5=sns.displot(data['Yearly Amount Spent'])

#### b. Bộ Data có các điểm nào cần chú ý (để thực hiện trong bước Data Processing)?
- Length of Membership có độ chênh lệch giữa max và min, tứ phân vị khá lớn. Cần chú ý tới biến này khi làm phân tích

In [ ]:
# percentile list
perc =[.25, .50, .75,.90]
  
# list of dtypes to include
include =[ 'float', 'int']
  
# calling describe method
desc = data.describe(percentiles = perc, include = include)
desc=desc.transpose() 
# display
desc

### 2. Multivariate Analysis:
Dưới đây là mô tả tương quan giữa 4 biến (x) với biến Yearly Amount Spent (y):
- x1 = Avg. Session Length
- x2 = Time on App
- x3 = Time on Website
- x4 = Length of Membership

#### a. Correlations
- Biểu đồ heatmap hiển thị tương quan giữa 4 biến trên với biến y
- Dựa vào thông số có giá trị luôn >=0 có thể thấy 4 biến trên có tương quan thuận với biến y. Nghĩa là khi giá trị của mỗi biến này tăng lên, y sẽ tăng theo một tỉ lệ nào đó.

In [ ]:
correlations=data.corr()
sns.heatmap(correlations)

#### b. Interaction
Để tìm hiểu chi tiết hơn về mối tương tác giữa mỗi biến x với y, chúng ta cùng xem các biểu đồ scatter bên dưới
- x1 có phân bổ khá đều, dựa vào đường line trend có thể nói biến x này không ảnh hưởng quá nhiều tới biến y
- x2,x3,x4 đều thể hiện rõ rằng khi giá trị các biến này tăng, biến y cũng tăng theo
- Đặc biệt ở biến x4 (Length of Membership), mức chênh lệch giữa min Y và Max Y theo x4 rất lớn. Chứng tỏ đây là biến có ảnh hưởng lớn nhất tới Y. Cần chú ý nhiều về biến này khi tiến hành phân tích và xây dựng mô hình.

In [ ]:
x1= sns.regplot(x="Time on Website", y="Yearly Amount Spent", data=data)

In [ ]:
sns.regplot(data=data,x='Time on App', y='Yearly Amount Spent')

In [ ]:
sns.regplot(data=data, x="Avg. Session Length", y="Yearly Amount Spent")

In [ ]:
sns.regplot(data=data,x="Length of Membership",y='Yearly Amount Spent')

---------------------------------------------------------------
## TODO 2: LINEAR REGRESSIONS

Xem lại Lab về Linear Regressions. Tìm hiểu thêm các notebook khác trên Kaggle.

Thực hiện bài toán Linear Regression với biến target (y) là Yearly Amount Spent. Lưu ý trình bày đầy đủ các bước:

Data Processing
Split Train & Test Data
Modeling
Evaluation

In [ ]:
data.columns=data.columns.str.replace(' ','_')
data.head()

In [ ]:
df=data.drop(columns=['Email','Address','Avatar'])
# df=df.loc[(df.Length_of_Membership)>=1]
df.head()

In [ ]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression().fit(x_train,y_train)

In [ ]:
coef=pd.DataFrame([x_train.columns,reg.coef_]).T
coef=coef.rename(columns={0:'Atributes',1:'Coefficients'})
coef

In [ ]:
ytest_pred=reg.predict(x_test)

In [ ]:
from sklearn import metrics
lin_acc=metrics.r2_score(ytest_pred,y_test)
print('R^2: ',lin_acc)
print('MAE: ',metrics.mean_absolute_error(ytest_pred,y_test))
print('MSE: ',metrics.mean_squared_error(ytest_pred,y_test))
print('RMSE: ',np.sqrt(metrics.mean_squared_error(ytest_pred,y_test)))
print('Max Error: ',metrics.max_error(ytest_pred,y_test))

In [ ]:
x_ax = range(len(x_test))
plt.plot(x_ax, y_test, lw=1, color="blue", label="original")
plt.plot(x_ax, ytest_pred, lw=0.8, color="red", label="predicted",marker="o", markersize=4)
plt.legend()
plt.show()

In [ ]:
df_plot = pd.DataFrame({'pred_amount_spend': ytest_pred, 'actual_amount_spend': y_test})
df_plot.head()

In [ ]:
sns.jointplot(x='pred_amount_spend',y='actual_amount_spend',data=df_plot,kind='reg')

-------------------------------------------------------
## TODO 3: CUSTOMER CLUSTERING
Tham khảo notebook sau: Clustering with K-Means

Áp dụng phương pháp Clustering lên bộ data để chia customers thành n nhóm (n tuỳ theo bạn lựa chọn)
#### 1. Trên n nhóm đã phân ra bằng clustering, trình bày chỉ số thông kê và phân phối của các biến sau:
    - Time on App
    - Avg. Session Length
    - Time on Website
    - Length of Membership

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)



In [ ]:
df

In [ ]:
features=['Avg._Session_Length','Time_on_App','Time_on_Website','Length_of_Membership']
X = df.loc[:,features]
Y=df.pop('Yearly_Amount_Spent')

#### 2. Visualize plot các yếu tố trên theo cặp. Ví dụ như bên dưới (cluster = các nhóm khác hàng đã chia)

In [ ]:
X.head()

In [ ]:
# Define a list of the features to be used for the clustering
kmeans = KMeans(n_clusters=4)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

Xy = X.copy()
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["Year_Amount"] = y
sns.relplot(
    x="value", y="Year_Amount", hue="Cluster", col="variable",
    height=4, aspect=1, facet_kws={'sharex': False}, col_wrap=4,
    data=Xy.melt(
        value_vars=features, id_vars=["Year_Amount", "Cluster"],
    ),
);

#### 3. Trình bày các quyết định, chiến lược kinh doanh có thể thực hiện dựa trên kết quả của clustering để giúp tăng doanh thu.